# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1QHOQt1J7d5Z4sBFKL9GN699OcPh2aVaP?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [51]:
#Passo 1: Pegar as cotações
#Passo 2: Importar a lista de produtos
#Passo 3: Recalcular o preço dos produtos
#Passo 4: Salvar os novos preços

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.edge.options import Options
from msedge.selenium_tools import Edge, EdgeOptions


#Abrir um navegador

options = EdgeOptions()
options.use_chromium = True
options.headless = True
driver = Edge(options = options)

driver = Edge()


driver.get("https://www.google.com.br/")
driver.find_element_by_xpath(
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("Cotação Dólar", Keys.ENTER)

cotação_dólar = driver.find_element_by_xpath(
    '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print(cotação_dólar)


driver.get("https://www.google.com.br/")
driver.find_element_by_xpath(
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("Cotação Euro", Keys.ENTER)

cotação_euro = driver.find_element_by_xpath(
    '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print(cotação_euro)


driver.get("https://www.melhorcambio.com/ouro-hoje#:~:text=O%20valor%20do%20grama%20do,em%20R%24%20280%2C60.")

cotação_ouro = driver.find_element_by_xpath(
    '//*[@id="comercial"]').get_attribute("value")
cotação_ouro = cotação_ouro.replace(",", ".")
print(cotação_ouro)

driver.quit()
#driver.get("https://www.bcb.gov.br/conversao")

 

4.8994
5.859167963
281.86


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [55]:
import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Ajuste,Preço Final
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [60]:
#Atualizar cotação
#Nas linhas onde na coluna "Moeda" = Dólar
tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cotação_dólar)
tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotação_euro)
tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cotação_ouro)


#Atualizar Preço Base Reais(Preço Base Original * Cotação)
tabela["Preço Base Reais"] = tabela["Preço Base Original"] * tabela["Cotação"]


#Atualizar Preço final(Preço Base Reais * Ajuste)
tabela["Preço Final"] = tabela["Preço Base Reais"] * tabela["Ajuste"]

display(tabela)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Ajuste,Preço Final
0,Câmera Canon,999.99,Dólar,4.899400,4899.351006,1.40,6859.091408
1,Carro Renault,4500.00,Euro,5.859168,26366.255833,2.00,52732.511667
2,Notebook Dell,899.99,Dólar,4.899400,4409.411006,1.70,7495.998710
3,IPhone,799.00,Dólar,4.899400,3914.620600,1.70,6654.855020
4,Carro Fiat,3000.00,Euro,5.859168,17577.503889,1.90,33397.257389
5,Celular Xiaomi,480.48,Dólar,4.899400,2354.063712,2.00,4708.127424
6,Joia 20g,20.00,Ouro,281.860000,5637.200000,1.15,6482.780000


### Agora vamos exportar a nova base de preços atualizada

In [62]:
#Salvar tabela atualizada
tabela.to_excel("Produtos Atualizados.xlsx", index = False)